In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import sys; sys.path.append(os.path.abspath(os.path.join('..'))); sys.path.append(os.path.abspath(os.path.join('.')))
from modules.random_fields import GaussianRF_idct
from tools import ConfigManager

In [2]:
from modules.act_norm import ActNorm

In [3]:
cm = ConfigManager('../config.yaml')
car = cm.get_model().to('cuda')

/home/st813@ad.eng.cam.ac.uk/benchmark_iib/modules/conv.py:13: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2425.)
  q, _ = torch.qr(weight)


In [4]:
car.load_state_dict(torch.load('../tmp/saved_ckpt/model_all_16_28.pth'))

<All keys matched successfully>

In [5]:
import matplotlib.pyplot as plt
import torch
import torchvision
def sample_grid(model, lowest, path=None, clip_range=None):
    plt.rcParams['figure.figsize'] = [8, 8]
    size_range = range(lowest, lowest+4*4, 4)
    for i, size in enumerate(size_range):
        plt.subplot(2, 2, i+1)
        plt.title(f'{size}x{size}')
        _sample_image_grid_from_model(model, resolution=size, device=torch.device('cuda'), clip_range=clip_range)
    if path is not None:
        plt.savefig(path)

@torch.no_grad()
def _sample_image_grid_from_model(model, device, resolution=28, clip_range=None):
    X_samples = model.sample(num_samples=4, resolution=resolution, device=device).cpu()
    if clip_range is not None:
        X_samples = torch.clamp(X_samples, clip_range[0], clip_range[1])
    print(X_samples.shape)
    grid = torchvision.utils.make_grid(X_samples, nrow=2)
    plt.imshow(grid[0], cmap="gray", vmin=0, vmax=255)

In [6]:
for i in car.flow:
    if isinstance(i, ActNorm):
        i.is_initialized = True
for i in car.ar_flow:
    if isinstance(i, ActNorm):
        i.is_initialized = True